In [66]:
import os
import requests
import re
from tqdm import tqdm
from urllib.parse import unquote
from bs4 import BeautifulSoup
caracSpe = ' '
aRemplacer = r'[<>:"/\\|?*#\[\]]'

In [2]:
# Les SuperVariables
URL = 'https://downloads.khinsider.com/game-soundtracks/album/outer-wilds-original-soundtrack'
DOSSIER = 'Outer Wilds'

In [3]:
rep = requests.get(URL)
soup = BeautifulSoup(rep.text, 'lxml')
if rep.ok:
  tracklist = soup.find('table', id='songlist')
  allA = tracklist.findAll('a')
  links = []
  for a in allA:
    if not 'https://downloads.khinsider.com' + a['href'] in links:
      links.append('https://downloads.khinsider.com' + a['href'])
else:
  print("problème à la page " + URL)
  print(rep)
print('les liens enregistré dans links')

les liens enregistré dans links


In [71]:
tracks = []
for link in tqdm(links):
  rep = requests.get(link)
  soup = BeautifulSoup(rep.text, 'lxml')
  if rep.ok:
    flac_url = False
    mp3_url = False
    for span in soup.findAll('span', class_='songDownloadLink'):
      if 'FLAC' in span.text:
        flac_url = span.parent['href']
      if 'MP3' in span.text:
        mp3_url = span.parent['href']
    if flac_url:
      tracks.append([flac_url, unquote(flac_url[re.search(r'\/(?!.*\/)', flac_url).end():])])
    elif mp3_url:
      tracks.append([mp3_url, unquote(mp3_url[re.search(r'\/(?!.*\/)', mp3_url).end():])])
    else:
      print('il n\'y a ni MP3 ni FLAC pour ' + link)
  else:
    print("problème à la page " + link)
    print(rep)
print('les liens des fichier audio sont dans tracks')

100%|██████████| 28/28 [00:15<00:00,  1.86it/s]

les liens des fichier audio sont dans tracks


In [70]:
if not os.path.exists('C:/Users/Jayma/Downloads/'+ DOSSIER):
  os.makedirs('C:/Users/Jayma/Downloads/'+ DOSSIER)
for track in tqdm(tracks):
  rep = requests.get(track[0])
  if rep.ok:
    if track[1] in os.listdir('C:/Users/Jayma/Downloads/'+ DOSSIER):
      print('doublon du fichier : ' + track[1] + ' ajout de autre')
      track[1] = 'autre ' + track[1]
    with open('C:/Users/Jayma/Downloads/'+ DOSSIER+ '/'+ track[1], 'wb') as f:
      f.write(rep.content)
  else:
    print("problème à la page " + track)
    print(rep)
print('tout est téléchargé de '+ DOSSIER)

  4%|▎         | 1/28 [00:02<01:19,  2.93s/it]

doublon du fichier : 01. Timber Hearth.flac ajout de autre


100%|██████████| 28/28 [01:25<00:00,  3.06s/it]

tout est téléchargé de Outer Wilds
